In [1]:
import tensorflow as tf

n_inputs = 28*28

n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

x = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'x')
y = tf.placeholder(tf.int64, shape = (None), name = 'y')

/home/odemakinde/cd/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
def neuron_layer(x, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        w = tf.Variable(init, name = 'kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name = 'bias')
        z = tf.matmul(x, w) + b
        if activation is not None:
            return activation(z)
        else:
            return z

In [7]:
import numpy as np
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(x, n_hidden1, name = 'hidden1', 
                          activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name = 'hidden2',
                          activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name = 'outputs')

In [8]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')

In [9]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
    training_op = optimizer.minimize(loss)

In [10]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [13]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [15]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [16]:
n_epochs = 40
batch_size = 50


In [18]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {x:x_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict = {x:x_batch, y:y_batch})
        acc_val = accuracy.eval(feed_dict = {x:mnist.validation.images, y:mnist.validation.labels})
        print(epoch, 'training accuracy: ', acc_train, 'val_accuracy: ', acc_val)
    save_path = saver.save(sess, './my_model_final.ckpt')

0 training accuracy:  0.94 val_accuracy:  0.914
1 training accuracy:  1.0 val_accuracy:  0.932
2 training accuracy:  0.92 val_accuracy:  0.9392
3 training accuracy:  0.84 val_accuracy:  0.947
4 training accuracy:  0.98 val_accuracy:  0.9528
5 training accuracy:  0.96 val_accuracy:  0.9572
6 training accuracy:  0.96 val_accuracy:  0.9582
7 training accuracy:  0.98 val_accuracy:  0.9624
8 training accuracy:  0.96 val_accuracy:  0.9646
9 training accuracy:  0.98 val_accuracy:  0.964
10 training accuracy:  0.94 val_accuracy:  0.9664
11 training accuracy:  0.98 val_accuracy:  0.9654
12 training accuracy:  0.96 val_accuracy:  0.9678
13 training accuracy:  1.0 val_accuracy:  0.9692
14 training accuracy:  0.98 val_accuracy:  0.9702
15 training accuracy:  0.98 val_accuracy:  0.9696
16 training accuracy:  1.0 val_accuracy:  0.9716
17 training accuracy:  0.98 val_accuracy:  0.9706
18 training accuracy:  0.98 val_accuracy:  0.9722
19 training accuracy:  1.0 val_accuracy:  0.9708
20 training accura

In [19]:
# implementing batch normalizatioin

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

x = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'x')

training = tf.placeholder_with_default(False, shape = (), name = 'training')

hidden1 = tf.layers.dense(x, n_hidden1, name = 'hidden1')
bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum = 0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = 'hidden2')
bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum = 0.9)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = 'outputs')
logits = tf.layers.batch_normalization(logits_before_bn, training = training, momentum = 0.9)


In [25]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
epoch = 10

init = ([tf.global_variables_initializer(), tf.local_variables_initializer()])
with tf.Session() as sess:
    init[0].run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], 
                    feed_dict = {training: True, x: x_batch, y: y_batch})
            accuracy_val = accuracy.eval(feed_dict = {x:mnist.test.images, 
                                                     y: mnist.test.labels})
            print(epoch, 'test accuracy: ', accuracy_val)

InvalidArgumentError: You must feed a value for placeholder tensor 'x' with dtype float and shape [?,784]
	 [[Node: x = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'x', defined at:
  File "/home/odemakinde/cd/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/odemakinde/cd/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/odemakinde/cd/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/odemakinde/cd/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/odemakinde/cd/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/odemakinde/cd/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).

In [ ]:
# implementing gradient clipping